# Rysk Finance

Defi options AMM

[Docs](shttps://docs.rysk.finance/getting-started/what-is-rysk)

Rysk options are priced using a AMM.


# Trading competition

[Trading comp](https://comp.rysk.finance/) is open until June.

# Data retrieval

We can retrieve data from the Rysk finance contracts from the follwoing [subgraph](https://api.goldsky.com/api/public/project_clhf7zaco0n9j490ce421agn4/subgraphs/devey/0.0.2/gn)

There is a depth of on chain data from the subgraph.

For example, we can retrieve all of the available markets as so;

In [ ]:
# install dependencies
!pip install pandas ccxt web3 .

In [ ]:
# we can retrieve the data using a simple client;
from dataclasses import dataclass
import requests
import json

# -H "Content-Type: application/json" -X POST -d 
@dataclass
class SubgraphClient:
    url: str
    
    
    def query(self, query):
        """Simple function to call a subgraph query."""
        headers = {
            "Content-Type": "application/json"
        }
        subgraph_query = {
            "query": query
        }
        response = requests.post(
            url=self.url,
            headers=headers,
            data=json.dumps(subgraph_query)
        )
        data = json.loads(response.content)['data']
        return data

In [ ]:
subgraph_url = "https://api.goldsky.com/api/public/project_clhf7zaco0n9j490ce421agn4/subgraphs/devey/0.0.2/gn"

subgraph_query = """
{series 
  {   
    id 
    expiration 
    netDHVExposure 
    strike
    isPut
    isBuyable
    isSellable
  }
}
"""

client = SubgraphClient(subgraph_url)
 
results = client.query(subgraph_query)

results['series'][:1]



In [ ]:
# we then parse into a pandas dataframe.

import pandas as pd
from datetime import datetime

# we need to convert the timestamp of the expiration to a date, we also need to clean up the decimals.
# conveniently the options expirations align with those from deribit;
# we format to allow easier look up later


price_devisor = 1_000_000_000_000_000_000
exposure_devisor = 100_000_000_000_000_0000

def from_timestamp(date_string):
    """Parse a timestamp."""
    return datetime.fromtimestamp(int(date_string))

def to_human_format(row):
    """
    Format the row to align to the ccxt unified client.
    'ETH-16MAY23-1550-C'
    """

    month_code = row.expiration_datetime.strftime("%b").upper()
    day = row.expiration_datetime.strftime("%d")
    year = str(row.expiration_datetime.year)[2:]
    strike_price = str(row.strike_price)

    return f"ETH-{day}{month_code}{year}-{strike_price}-{'P' if row.isPut else 'C'}"


df = pd.DataFrame(results['series'])


# We format the data
df['expiration_datetime'] = df["expiration"].apply(from_timestamp)
df['strike_price'] = df['strike'].apply(lambda x: int(int(x) / price_devisor))
df['net_DHV_exposure'] = df['netDHVExposure'].apply(lambda x: int(int(x) / exposure_devisor))
df["human_strike"] = df.apply(to_human_format, axis=1)




# we filter out all markets that have already closed;
df = df[df["expiration_datetime"] > datetime.now()]



# we filter out markets that we cant trade at all
df = df[df["isBuyable"] | df['isSellable']]


# We only display our interested columns
columns = ["human_strike", "expiration_datetime", "strike_price", "net_DHV_exposure", "isPut", "isBuyable", "isSellable"]
df[columns].sort_values(["expiration_datetime", "strike_price"]).head(50)


In [ ]:
def refresh_df():
    


    results = client.query(subgraph_query)
    df = pd.DataFrame(results['series'])


    # We format the data
    df['expiration_datetime'] = df["expiration"].apply(from_timestamp)
    df['strike_price'] = df['strike'].apply(lambda x: int(int(x) / price_devisor))
    df['net_DHV_exposure'] = df['netDHVExposure'].apply(lambda x: int(int(x) / exposure_devisor))
    df["human_strike"] = df.apply(to_human_format, axis=1)




    # we filter out all markets that have already closed;
    df = df[df["expiration_datetime"] > datetime.now()]



    # we filter out markets that we cant trade at all
    df = df[df["isBuyable"] | df['isSellable']]

    return df

df = refresh_df()

# Exposure Arbitrages

Conceptually, the DHV is pricing its options based on demand, where by the price is a function of the expected volitity based on the BlackSholes formula combined with net exposure at each strike.

We can therefore seek to find a number of arbitrages within the markets;

We will initially demonstrate a function to identify arbitrage based on short term options versus long term options.

As a basic strategy, we will look at each side of each of the markets. For each of the tradable options, we will look for circumstances whereby the option may be immediately sold to either capitalise on the differential between the price, or the differentical between the time.


In [ ]:
def does_arb_exist(row, display=False):
    """We will check the inital dataframe."""
    # find similar candidates
    candidates = df[df.isPut == row.isPut]
    
    # we filter out older markets
    candidates = candidates[candidates.expiration_datetime >= row.expiration_datetime]
    
    if row.isBuyable:
        candidates = candidates[candidates.isSellable]
    
    if row.isSellable:
        candidates = candidates[candidates.isBuyable]
    
    if row.isPut:
        # we want to look for puts which have a strike greater than our current row.
        candidates = candidates[candidates.strike >= row.strike]
        if row.isBuyable:
            # we are looking to *buy* cheap puts then sell them on.
            # expensive options are those where the dhv has has sold a lot at that strike 
            # and is therefore increasing the price to compensate. 
            # This is denoted with a negative exposure
            
            # our candidates will be those with new_dhv_exposure > our row 
            candidates = candidates[candidates.net_DHV_exposure > row.net_DHV_exposure]
            candidates.sort_values("net_DHV_exposure")
            
        else:
            # We now are looking to *sell* expensive puts 
            # and then buy cheap ones
            candidates = candidates[candidates.net_DHV_exposure < row.net_DHV_exposure]
            
    
    else:
        # we want to look for call which have a strike lower than our current row
        candidates = candidates[candidates.strike <= row.strike]
        if row.isBuyable:
            # we are looking to *buy* cheap puts then sell them on.
            # expensive puts are those where the dhv has has sold a lot at that strike 
            # and is therefore increasing the price to compensate. 
            # This is denoted with a negative exposure
            
            # our candidates will be those with new_dhv_exposure > our row 
            candidates = candidates[candidates.net_DHV_exposure > row.net_DHV_exposure]
        
        else:
            # We now are looking to *sell* expensive puts 
            # and then buy cheap ones
            candidates = candidates[candidates.net_DHV_exposure < row.net_DHV_exposure]
        
    def _format_row(row):
        """Format the row."""
        return f"{row.human_strike} @ {row.net_DHV_exposure}"
    

        
        
    if len(candidates):
        
#       # we now look for the biggest differences between the candidates and the row. 
        # we return this so we further can sort.
        
        candidates['value'] = abs(candidates.net_DHV_exposure - row.net_DHV_exposure)
        
        candidates = candidates.sort_values('value')
        if display:
            print(f"Potential Arb between: {_format_row(row)} & {_format_row(candidates.iloc[0])}")


    return False


df['arbs'] = df.apply(does_arb_exist, axis=1)
top_arbs = df.sort_values("arbs", ascending=False).head()


top_arbs.apply(does_arb_exist, axis=1, args=(True,))

In [ ]:
df

# Price Retrieval 

Prices are extracted from the [Beyond Pricer](https://goerli.arbiscan.io/address/0xc939df369C0Fc240C975A6dEEEE77d87bCFaC259#readContract)

This allows us to take the id retrieved from the subgraph and use it to extract all of the prices, (Unfortunately this is iteratively, however this could be improved by implementing and deploying a smart contract which would collect the data in one call)

In [5]:
!pip install .

Processing /home/tom/Desktop/Fun/rysk_examples
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


  Created wheel for rysk-client: filename=rysk_client-0.1.0-py3-none-any.whl size=9431 sha256=3ce2ce9a0407f078122244efe5b182e52bf9c663df69aa7e6de21caafe0fbc6a
  Stored in directory: /tmp/pip-ephem-wheel-cache-1r6yr07k/wheels/d0/5d/c5/06071ec2d9be7dfabf83e13d497e7bd4b06d0041d896c67705
Successfully built rysk-client
  Attempting uninstall: rysk-client
    Found existing installation: rysk-client 0.1.0
    Uninstalling rysk-client-0.1.0:
      Successfully uninstalled rysk-client-0.1.0
You should consider upgrading via the '/home/tom/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [12]:

from rysk_client.src.utils import get_contract, get_web3

w3 = get_web3()

option_catalogue = get_contract("option_catalogue", w3)
expirations = option_catalogue.functions.getExpirations().call()


print(f"Expirations on {len(expirations)} dates.")



Expirations on 6 dates.


In [7]:
# we now need to get all the option details
# 
from dataclasses import dataclass
@dataclass
class RyskMarket:
    expiration: int
    strike: int
    is_put: bool

markets = []
for expiration in expirations:
    put_strikes = option_catalogue.functions.getOptionDetails(expiration, True).call()
    call_strikes = option_catalogue.functions.getOptionDetails(expiration, False).call()
    
    for strike in put_strikes:
        markets.append(RyskMarket(expiration, strike, True))
        
    for strike in call_strikes:
        markets.append(RyskMarket(expiration, strike, False))
markets

[RyskMarket(expiration=1683878400, strike=1700000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=1800000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=1900000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=2000000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=2100000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=1725000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=1750000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=1775000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=1825000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=1850000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=1875000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strike=1925000000000000000000, is_put=True),
 RyskMarket(expiration=1683878400, strik

In [16]:
# now we have the markets, we need to to get the otkens

option_registry = get_contract("option_registry", w3)

In [15]:
# we check approvals now

def check_approval(spender, amount, address):
    """Check the approval of the spender."""
    
weth_contract = get_contract("weth", w3)

FileNotFoundError: [Errno 2] No such file or directory: 'contracts/packages/contracts/token/ERC20/ERC20.sol/ERC20.json'

In [ ]:
Collateral.eth.value, Collateral.usdc.value

In [ ]:
from enum import Enum

def filter_by_human_format(df, human_format):
    """
    Filter the DataFrame based on the human-readable format and return a single row.
    """

    filtered_option = df[df["human_strike"] == human_format]
    if len(filtered_option) > 0:
        return filtered_option.iloc[0]  # Return the first row if found
    else:
        return None  # Return None if no matching row is found



def get_options_prices(market, amount=1000000000000000000, side="buy", collateral="eth"):
    """, 
    We call the beyond pricer to determine the prices for a market
    huge thanks to 0xPawel2 and Jib &&
    """
    if side not in ["buy", "sell"]:
        raise ValueError("Side must be buy or sell")

    option_data = filter_by_human_format(df, market)
    if option_data is None:
        raise TypeError("Unable to find the market")

    if not Collateral.is_supported(collateral):
        raise TypeError(f"Collateral {collateral} is not supported")
    # here we call the contract functions

    option_series = (
        int(option_data['expiration']),
        int(option_data['strike']),
        bool(option_data['isPut']),
        '0x3b3a1dE07439eeb04492Fa64A889eE25A130CDd3', 
        '0x408c5755b5c7a0a28D851558eA3636CfC5b5b19d', 
        '0x408c5755b5c7a0a28D851558eA3636CfC5b5b19d'
    )
#     print(option_series)
    result = contract_instance.functions.quoteOptionPrice(
        option_series,
        int(amount),
        True if side == "sell" else False,
        int(option_data['netDHVExposure'])).call()
    # totalPremium
    # totalDelta
    # totalFees
    return result[0] # / 1_000_000

df["ask"] = df.human_strike.apply(get_options_prices)
df['bid'] = df.human_strike.apply(get_options_prices, side="sell")

df[columns + ["ask", "bid"]].head()

In [ ]:
# Unified crypto client.

import ccxt
# we filter out everything except ethereum options


market = df.iloc[0].human_strike
exchange = ccxt.deribit()
markets = exchange.fetch_markets()



ticker = exchange.fetch_ticker(market)


print(f"{market}: bid: {ticker['bid']} ask: {ticker['ask']}")

In [8]:
# we need a function to convert from eth to USD to align to rysk trades

price = exchange.fetch_ticker("ETH/USDC")['last']

def get_price_from_cex(market):
    """Get bid and ask from the exchange."""
    try:
        ticker = exchange.fetch_ticker(market)
    except:
        return None
    bid, ask = ticker['bid'], ticker['ask']
    
    if bid is not None:
        bid = price * bid
    if ask is not None:
        ask = price * ask
    return bid, ask
df['cex'] = df.human_strike.apply(get_price_from_cex)
df.dropna(inplace=True)
df['db_bid'] = df['cex'].apply(lambda x: x[0])
df['db_ask'] = df['cex'].apply(lambda x: x[1])

df.dropna(inplace=True)



NameError: name 'exchange' is not defined

In [ ]:
df['arb_gain'] = (df['bid'] - df['db_ask'] ) /  df['db_ask'] * 100

df[columns + ["bid", "db_ask", "arb_gain"]].sort_values("arb_gain", ascending=False)

In [ ]:
df

# Positions

In order to retrieve a traders position from the DHV, we are limited in the fact that we are retrieveing the traders position on each market as opposed to retrieving the position in aggregate.

Note, this has implication in terms of the liquidations of the positions held by traders.



In [ ]:
df.iloc[0]


# Redeeming & Settling Positions

In [ ]:
# Settling the option occurs when a user is short the option and would like to sell back to the pool.

contract = get_contract("opyn_controller")

In [ ]:
# we first need to get our positions, which can be done using the following subgraph query;

In [ ]:
contract.all_functions()

In [ ]:
owner_address = "0x9B8a204636a7aa9c33053d9C3A828720d32212e8"

# for each vault, we need to parse the results of the folling contract call;


class PositionSide(Enum):
    long = "long"
    short = "short"
    


class Position():
    side: PositionSide
    amount: float
    collateral: str = "usdc"
    pnl: float

def get_short_position(vault_id):
    """Create a position representation from the contract calls"""
    contract = get_contract("opyn_controller")
    res = contract.functions.getVaultWithDetails(owner_address, vault_id).call()
    position = res[0]
    amount_short = position[3][0]
    collateral = position[5][0]
    market_address = position[0][0]
    print(f"otoken market: {market_address}")
    print(f"amount short: {int(amount_short/1_000_000_00)} eth")
    print(f"Collateral: ${collateral/1_000_000}")
    return market_address, amount_short, collateral
    
vault_id = 4
position = get_short_position(vault_id=vault_id)
position


In [ ]:
# we now need to check if the market has expired;
contract = get_contract("opyn_controller")
contract.functions.hasExpired(position[0]).call()

In [ ]:
position

In [ ]:
contract.functions.isSettlementAllowed(position[0]).call()

In [ ]:
contract.functions.getProceed(owner_address, vault_id).call() / 1_000_000

In [ ]:
contract.functions.getPayout(position[0], position[1]).call()

In [ ]:
from rysk_client.src.action_types import ActionType
from rysk_client.src.constants import NULL_ADDRESS

# attempting to settle

operate_tuple = [{
    "actionType": ActionType.SettleVault.value,
    "amount": 0,
    "asset": NULL_ADDRESS,
    "data": NULL_ADDRESS,
    "index": 0,
    "owner": w3.toChecksumAddress(owner_address),
    "secondAddress": w3.toChecksumAddress(owner_address),
    "vaultId": 1,
}]
print("Attempting operation")
print(operate_tuple)


func = contract.functions.operate(operate_tuple).buildTransaction({"from": owner_address})
print(func)


In [ ]:
# Now we have the settle transaction


# Creating Orders

[Sell option tx](
https://goerli.arbiscan.io/tx/0x17c34633ffabfb306094841f0e31d97d0b4d3d177d4ef8ebb084d7f9f51d6439)

[Buy option tx](https://goerli.arbiscan.io/tx/0x3e3c80647170b48dbbb75e291c9dee05d82bc24573658380107913c2f8feb863)

In [ ]:
from rysk_client.src.constants import SUPPORTED_LEVERAGES, NULL_ADDRESS

contract = get_contract("option_exchange")

new_vault_id = 8
owner_address = w3.toChecksumAddress(owner_address)


class UnsupportedLeverageException(Exception):
    """Unsupported leverage selected"""

def sell_option(amount=1, leverage=1):
    if leverage not in SUPPORTED_LEVERAGES:
        raise UnsupportedLeverageException(f"{leverage} is not supported!")
    
    
    
    operate_tuple = [
      {
        "operation": 0,
        "operationQueue": [
          {
            "actionType": ,
            "owner": w3.toChecksumAddress("0x9b8a204636a7aa9c33053d9c3a828720d32212e8"),
            "secondAddress": w3.toChecksumAddress("0xb672fe86693bf6f3b034730f5d2c77c8844d6b45"),
            "asset": w3.toChecksumAddress("0x408c5755b5c7a0a28d851558ea3636cfc5b5b19d"),
            "vaultId": 5,
            "amount": 817283000,
            "optionSeries": {
              "expiration": 1,
              "strike": 1,
              "isPut": True,
              "underlying": NULL_ADDRESS,
              "strikeAsset": NULL_ADDRESS,
              "collateral": NULL_ADDRESS
            },
            "indexOrAcceptablePremium": 0,
            "data": "0x0000000000000000000000000000000000000000"
          },
          {
            "actionType": 1,
            "owner": w3.toChecksumAddress("0x9b8a204636a7aa9c33053d9c3a828720d32212e8"),
            "secondAddress": w3.toChecksumAddress("0xb672fe86693bf6f3b034730f5d2c77c8844d6b45"),
            "asset": w3.toChecksumAddress("0x1f2ba8c9ac6721208800d16cca7af1e4deaf428d"),
            "vaultId": 5,
            "amount": 100000000,
            "optionSeries": {
              "expiration": 1,
              "strike": 1,
              "isPut": True,
              "underlying": NULL_ADDRESS,
              "strikeAsset": NULL_ADDRESS,
              "collateral": NULL_ADDRESS
            },
            "indexOrAcceptablePremium": 0,
            "data": "0x0000000000000000000000000000000000000000"
          }
        ]
      },
      {
        "operation": 1,
        "operationQueue": [
          {
            "actionType": 2,
            "owner": NULL_ADDRESS,
            "secondAddress": owner_address,
            "asset": NULL_ADDRESS,
            "vaultId": 0,
            "amount": 1000000000000000000,
            "optionSeries": {
              "expiration": 1686297600,
              "strike": 2000000000000000000000,
              "isPut": False,
              "underlying": w3.toChecksumAddress("0x3b3a1de07439eeb04492fa64a889ee25a130cdd3"),
              "strikeAsset": w3.toChecksumAddress("0x408c5755b5c7a0a28d851558ea3636cfc5b5b19d"),
              "collateral": w3.toChecksumAddress("0x408c5755b5c7a0a28d851558ea3636cfc5b5b19d")
            },
            "indexOrAcceptablePremium": 51017398,
            "data": "0x0000000000000000000000000000000000000000"
          }
        ]
      }
    ]

    # sell tx
#     print(operate_tuple)

    func = contract.functions.operate(operate_tuple).buildTransaction({"from": owner_address})
#     print(func)
    return func

sell_option()


In [ ]:
from rysk_client.src.rysk_option_market import RyskOptionMarket


owner_address = w3.toChecksumAddress(owner_address)
markets = {}

def to_market(row):
    markets[row.human_strike] = RyskOptionMarket(
        name=row.human_strike,
        strike=int(row.strike),
        expiration=int(row.expiration),
        is_put=row.isPut
    )
    

        

underlying = w3.toChecksumAddress("0x3b3a1de07439eeb04492fa64a889ee25a130cdd3")
strike_asset = w3.toChecksumAddress("0x408c5755b5c7a0a28d851558ea3636cfc5b5b19d")
collateral = w3.toChecksumAddress("0x408c5755b5c7a0a28d851558ea3636cfc5b5b19d")

def buy_option(market: str, amount: float):
    """
    
    """
    
    rysk_option_market = markets[market]
    
    _amount = amount * 1000000000000000000
    
    acceptable_premium = get_options_prices(
        market, 
        amount=_amount,
        side="buy",
        collateral="eth")
    
    operate_tuple = [
      {
        "operation": 1,
        "operationQueue": [
          {
            "actionType": ActionType.OpenVault.value,
            "owner": NULL_ADDRESS,
            "secondAddress": NULL_ADDRESS,
            "asset": NULL_ADDRESS,
            "vaultId": 0,
            "amount": 0,
            "optionSeries": {
              "expiration": int(rysk_option_market.expiration),
              "strike": int(rysk_option_market.strike),
              "isPut": bool(rysk_option_market.is_put),
              "underlying": underlying,
              "strikeAsset": strike_asset,
              "collateral": collateral
            },
            "indexOrAcceptablePremium": 0,
            "data": "0x0000000000000000000000000000000000000000"
          },
          {
            "actionType": ActionType.MintShortOption.value,
            "owner": NULL_ADDRESS,
            "secondAddress": owner_address,
            "asset": NULL_ADDRESS,
            "vaultId": 0,
            "amount": int(_amount),
            "optionSeries": {
              "expiration": rysk_option_market.expiration,
              "strike": int(rysk_option_market.strike),
              "isPut": rysk_option_market.is_put,
              "underlying": underlying,
              "strikeAsset": strike_asset,
              "collateral": collateral
            },
            "indexOrAcceptablePremium": int(acceptable_premium * 1.1),
            "data": "0x0000000000000000000000000000000000000000"
          }
        ]
      }
    ]


    # buy tx
#     print(operate_tuple)
    
    contract = get_contract("option_exchange")
    func = contract.functions.operate(operate_tuple).buildTransaction({"from": owner_address})
    # print(func)
    return func

df = refresh_df()
df.apply(to_market, axis=1)

amount = 1

market = "ETH-09JUN23-2100-C"

tx = buy_option(market=market, amount=amount)

print(tx)
        

In [ ]:
# generalised sell


new_vault_id = 6
owner_address = w3.toChecksumAddress(owner_address)


JUN30_call_weth_collateral =  w3.toChecksumAddress("0xa1eabe40dab7ceeb08fe5d2fca0561091d5211d4")

def sell_option(
    market: str,
    amount: float,
    collateral_asset: str="eth",
    leverage: float=1):
    
    if collateral_asset not in Collateral.__members__:
        raise ValueError("Collateral Not Supported")
    
    collateral = Collateral(collateral_asset)
    option_exchange_address = w3.toChecksumAddress(get_contract("option_exchange").address)

    rysk_option_market = markets[market]
        
    _amount = amount * 1000000000000000000
    
    acceptable_premium = get_options_prices(
        market, 
        amount=_amount,
        side="sell",
        collateral="eth")

    
    position_id = 7
    vault_id = position_id
    
    collateral =  WETH_ERC20
    
    # is this the option series??
    option_otoken = JUN30_call_weth_collateral
    
    
    
    underlying = WETH_ERC20
    
    strike_asset = USDC_ERC20
    
    operate_tuple = [
        
      {
        "operation": 0,
        "operationQueue": [
          {
            "actionType": ActionType.DepositCollateral.value,
            "owner": owner_address,
            "secondAddress": option_exchange_address,
            "asset": collateral,
            "vaultId": vault_id,
            "amount": _amount,
            "optionSeries": {
              "expiration": 1,
              "strike": 1,
              "isPut": True,
              "underlying": NULL_ADDRESS,
              "strikeAsset": NULL_ADDRESS,
              "collateral": NULL_ADDRESS
            },
            "indexOrAcceptablePremium": 0,
            "data": "0x0000000000000000000000000000000000000000"
          },
          {
            "actionType": ActionType.MintShortOption.value,
            "owner": owner_address,
            "secondAddress": option_exchange_address,
            "asset": option_otoken,
            "vaultId": vault_id,
            "amount": 100000000,
            "optionSeries": {
              "expiration": 1,
              "strike": 1,
              "isPut": True,
              "underlying": NULL_ADDRESS,
              "strikeAsset": NULL_ADDRESS,
              "collateral": NULL_ADDRESS
            },
            "indexOrAcceptablePremium": 0,
            "data": "0x0000000000000000000000000000000000000000"
          }
        ]
      },
      {
        "operation": 1,
        "operationQueue": [
          {
            "actionType": ActionType.BurnShortOption.value,
            "owner": NULL_ADDRESS,
            "secondAddress": owner_address,
            "asset": NULL_ADDRESS,
            "vaultId": 0,
            "amount": _amount,
            "optionSeries": {
              "expiration": rysk_option_market.expiration,
              "strike": rysk_option_market.strike,
              "isPut": rysk_option_market.is_put,
              "underlying": underlying,
              "strikeAsset": strike_asset,
              "collateral": collateral
            },
            "indexOrAcceptablePremium": int(108230854 * 0.9),
            "data": "0x0000000000000000000000000000000000000000"
          }
        ]
      }
    ]

    # sell tx
    from pprint import pprint
    pprint(operate_tuple)

    func = contract.functions.operate(operate_tuple).buildTransaction({"from": owner_address})
#     print(func)
    return func



df = refresh_df()
df.apply(to_market, axis=1)

amount = 1
market = "ETH-30JUN23-2200-C"



for i in markets:
    print(f"Processing sell {i} for {amount}")
    try:
        
        tx = sell_option(market=i, amount=1)
    except Exception as error:
        print(error)
        
    break



In [ ]:
def get_otoken_from_market():
    raise NotImplemented
    

    
# we want to 

In [ ]:
sell_option(market="ETH-09JUN23-1800-P", amount=1)

In [ ]:
private_key = "0x75cc9212e9e1243b9a3e5db5012f39469254088e33363324ad94dd0b212d7efa"


In [ ]:
def await_for_execution(tx):
    nonce = w3.eth.get_transaction_count(owner_address)
    tx['nonce'] = nonce
    signed_txn = w3.eth.account.sign_transaction(tx, private_key=private_key)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
    print(f"Submitted buy option tx")
    print(str(tx_hash.hex()))
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash, 180)
    print(f"Settled.")
    return receipt

df = refresh_df()
df.apply(to_market, axis=1)

amount = 5

successful = []
failed = []
while True:
    for i in markets:
        if i.find("30JUN") >= 0: continue
        print(f"Processing buy {i} for {amount}")
        try:
            tx = buy_option(market=market, amount=amount)
            receipt = await_for_execution(tx)
            if receipt.status:
                successful.append({i: receipt})
            else:
                failed.append({i: receipt})
        except Exception as error:
            print(error)
    break
    time.sleep(10)
    df = refresh_df()
    df.apply(to_market, axis=1)



In [ ]:
# TODO: re eqaluate the function to determine if the option is tradable. 

In [ ]:
def check_approvals(spender, amount):
    

In [13]:
w3.toChecksumAddress("0xb672fe86693bf6f3b034730f5d2c77c8844d6b45")

'0xb672fE86693bF6f3b034730f5d2C77C8844d6b45'